# NRPyLaTeX Tutorial Notebook

In [1]:
!pip install nrpylatex~=1.2 > /dev/null
!pip freeze | grep nrpylatex

from nrpylatex import parse_latex
%load_ext nrpylatex.extension

nrpylatex @ git+https://github.com/zachetienne/nrpylatex.git@aee228bb20652da4c650e2958673fdabf2b88763


## Example (1). Schwarzschild Solution to Einstein's Equations

In [2]:
%%parse_latex --reset

%% define Schwarzschild coordinate system
% coord [t, r, \theta, \phi]

%% define any relevant constant(s)
% define G M --const

%% initialize every component of the metric to zero
% define gDD --dim 4 --zero

%% define Schwarzschild metric diagonal
% g_{t t} &= -\left(1 - \frac{2GM}{r}\right) \\
%% e.g. g_{t t} = g_{0 0} (= g_{x_0 x_0})
% g_{r r} &=  \left(1 - \frac{2GM}{r}\right)^{-1} \\
% g_{\theta\theta} &= r^2 \\
% g_{\phi\phi} &= r^2 \sin^2\theta \\

%% generate inverse gUU, determinant det(gDD), and connection GammaUDD
% assign gDD --metric

% ignore "\begin{align}" "\end{align}"
\begin{align}
    %% define Riemann Tensor, Ricci Tensor, Ricci Scalar
    R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu}
        + \Gamma^\alpha_{\mu\gamma} \Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma} \Gamma^\sigma_{\beta\mu} \\
    R_{\beta\nu} &= R^\alpha{}_{\beta\alpha\nu} \\
    R &= g^{\beta\nu} R_{\beta\nu} \\

    %% define Einstein Tensor and Kretschmann Scalar
    G_{\beta\nu} &= R_{\beta\nu} - \frac{1}{2} R g_{\beta\nu} \\
    K &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu} %% automatic index raising (lowering)
\end{align}

('gdet',
 'R',
 'theta',
 'gUU',
 'epsilonUUUU',
 'GDD',
 'gDD',
 'RDDDD',
 'RUUUU',
 'RDD',
 'M',
 'r',
 'RUDDD',
 'G',
 'K',
 'GammaUDD')

In [3]:
from sympy import simplify

simplify(K)

48*G**2*M**2/r**6

## Example (2). BSSN Hamiltonian Constraint for Vacuum Spacetimes

In [4]:
%%parse_latex --reset

%% define 3-dimensional, unspecified metric
%% append suffix _dD to every partial derivative
% define gammabarDD --dim 3 --deriv dD --metric

%% apply symmetry [i][j] == [j][i] to each variable
% define AbarDD RbarDD --dim 3 --deriv dD --sym sym01

%% parse equation internally (avoid LaTeX rendering)
% \bar{R} = \bar{\gamma}^{ij} \bar{R}_{ij}

%% note, gradient of cf must have a dimension
% define cf --dim 3 --deriv dD

%% perform string replacement (with a capture group)
% srepl "e^{-4\phi}" -> "\text{cf}^2"
% srepl "\partial_<1..> \phi" -> "\partial_<1..> \text{cf} \frac{-1}{2 \text{cf}}" --persist

% ignore "\begin{equation}" "\end{equation}"
\begin{equation}
    %% define custom operator \bar{D}^2
    % srepl "\bar{D}^2" -> "\bar{D}^i \bar{D}_i"
    H = \frac{2}{3} K^2 - \bar{A}_{ij} \bar{A}^{ij} + e^{-4\phi}
        \left(\bar{R} - 8 \bar{D}^i \phi \bar{D}_i \phi - 8 \bar{D}^2 \phi\right)
\end{equation}

('GammabarUDD',
 'H',
 'cf_cdbarD_dD',
 'cf_dDD',
 'phi',
 'phi_cdbarU',
 'AbarDD',
 'gammabarDD',
 'phi_cdbarD',
 'cf',
 'gammabarDD_dD',
 'K',
 'Rbar',
 'phi_cdbarDU',
 'cf_cdbarD',
 'RbarDD',
 'epsilonUUU',
 'gammabarUU',
 'AbarUU',
 'cf_dD',
 'gammabardet',
 'phi_cdbarDD')

## Example (3). Exception Handling and Debugging

In [5]:
%%parse_latex

% define gUD vD --dim 3
v^a = g^a_b v_b

TensorError: illegal bound index 'b' in vU


In [6]:
%%parse_latex

% define gUU --dim 3
v^a = g^{cb} v_b

TensorError: unbalanced free index {'a', 'c'} in vU


In [7]:
%parse_latex v^a = g^{a*} v_b

ParseError: v^a = g^{a*} v_b
                      ^
unexpected '*' at position 91


In [8]:
%parse_latex J^a = (4\pi k)^{-1} \nabla_b F^{ab}

ParseError: J^a = (4\pi k)^{-1} \nabla_b F^{ab}
                                ^
cannot generate covariant derivative without defined metric 'g'


Note, the `parse_latex()` function (**not** the magic command) will display a stacktrace.

In [9]:
parse_latex(r"""
    % srepl "<1>^{<2..>} \partial_{<2..>}" -> "<1>^{<2..>} % deriv dupD \partial_{<2..>}" --persist

    % define betaU ADD --dim 3 --deriv dD
    B_{ij} = \mathcal{L}_\beta A_{ij}
""", reset=True, debug=True)

> LaTeX Input [1]
  \mathcal{L}_\text{beta} A_{i j} = \text{beta}^{i_1} % deriv dupD \partial_{i_1} A_{i j} + (\partial_{i} \text{beta}^{i_1}) A_{i_1 j} + (\partial_{j} \text{beta}^{i_1}) A_{i i_1}
< SymPy Output
  Tensor(ADD_ldbeta, i, j) = Tensor(betaU, i_1)*Tensor(ADD_dupD, i, j, i_1) + Tensor(ADD, i, i_1)*Tensor(betaU_dD, i_1, j) + Tensor(ADD, i_1, j)*Tensor(betaU_dD, i_1, i)
< Python Output
  ADD_ldbeta = [[Add(sum(Mul(betaU[i_1], ADD_dupD[i][j][i_1]) for i_1 in range(3)), sum(Mul(ADD[i][i_1], betaU_dD[i_1][j]) for i_1 in range(3)), sum(Mul(ADD[i_1][j], betaU_dD[i_1][i]) for i_1 in range(3))) for j in range(3)] for i in range(3)]

> LaTeX Input [2]
  B_{i j} = \mathcal{L}_\beta A_{ij}
< SymPy Output
  Tensor(BDD, i, j) = Tensor(ADD_ldbeta, i, j)
< Python Output
  BDD = [[ADD_ldbeta[i][j] for j in range(3)] for i in range(3)]



('ADD_ldbeta', 'betaU_dD', 'ADD_dupD', 'BDD', 'betaU', 'ADD')

## Example (4). A Brief Survey of NRPyLaTeX's Features

In [10]:
# Expression Parsing
parse_latex(r"""
    \frac{2}{3} - 2\sqrt[5]{x + 3}
""", reset=True)

2/3 - 2*(x + 3)**(1/5)

In [11]:
# Comma/Colon Notation
parse_latex(r"""
    % define vD uD wD --dim 3 --deriv dD
    T_{abc} = ((v_a + u_a)_{,b} - w_{a,b})_{,c}
""", reset=True)
print(TDDD[0][0][0])

uD_dDD000 + vD_dDD000 - wD_dDD000


In [12]:
# Mixed Dimension Indexing
parse_latex(r"""
    % define TUU --dim 3
    % define vD  --dim 2
    % index i --dim 2
    w^a = T^{a i} v_i
""", reset=True)
wU

[TUU01*vD0 + TUU02*vD1, TUU11*vD0 + TUU12*vD1, TUU21*vD0 + TUU22*vD1]

In [13]:
# Change Index Loop/Summation Range
parse_latex(r"""
    % coord [\theta, \phi]
    % index [a-z] --dim 2
    % define r --const

    % define gDD --dim 2 --zero
    g_{0 0} = r^2 \\
    g_{1 1} = r^2 \sin^2(\theta)
    % assign gDD --metric

    R^\alpha_{\beta\mu\nu} = \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu}
        + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
    R_{\alpha\beta\mu\nu} = g_{\alpha a} R^a_{\beta\mu\nu} \\
    R_{\beta\nu} = R^\alpha_{\beta\alpha\nu} \\
    R = g^{\beta\nu} R_{\beta\nu}
""", reset=True)

from IPython.display import display
display(GammaUDD[0][1][1])
display(R)

-sin(theta)*cos(theta)

2/r**2

In [14]:
# Disable Implied Summation and
# Change Derivative Suffix Inline (e.g. Upwind)
parse_latex(r"""
    % coord [r, \theta, \phi]

    % define gammahatDD --dim 3 --zero
    % define RDD --dim 3 --deriv dD
    % \hat{\gamma}_{ii} = R_{ii} % noimpsum

    % define hDD --dim 3 --deriv dD
    % \bar{\gamma}_{ij} = h_{ij} R_{ij} + \hat{\gamma}_{ij} % noimpsum

    T_{ijk} = \partial_k % deriv dupD
        \bar{\gamma}_{ij}
""", reset=True)

('gammahatDD', 'gammabarDD', 'hDD', 'RDD', 'TDDD')

In [15]:
# Multiple Metric Tensors
parse_latex(r"""
    % define k --const

    % define FUU --dim 4 --deriv dD --sym anti01
    % define gDD --dim 4 --deriv dD --metric
    % define ghatDD --dim 4 --deriv dD --metric

    J^\mu = (4\pi k)^{-1} \hat{\nabla}_\nu F^{\mu\nu}
""", reset=True)

('ghatDD_dD',
 'gdet',
 'GammahatUDD',
 'JU',
 'ghatDD',
 'gUU',
 'gDD_dD',
 'epsilonUUUU',
 'gDD',
 'ghatdet',
 'FUU_dD',
 'ghatUU',
 'k',
 'FUU_cdhatD',
 'FUU',
 'GammaUDD')

In [16]:
# Assign Specific Metric to Tensor
parse_latex(r"""
    % define k --const

    % define hDD --dim 4 --deriv dD --metric
    % define FUU --dim 4 --deriv dD --sym anti01 --metric hDD

    J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
""", reset=True)

('JU',
 'epsilonUUUU',
 'hDD',
 'FUU_dD',
 'k',
 'hDD_dD',
 'FUU',
 'FUU_cdD',
 'hdet',
 'hUU',
 'GammaUDD')